# Labs organized for Castor upload

### This lab layout follows the request of scientists and Castor staff  for upload in Castor

In [ ]:
#import libraries

In [ ]:
import os
import sys
from functools import reduce
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import ipywidgets as widgets

sys.path.insert(0, '../../')
from brickstudy import foundation

In [ ]:
### import example lab organization from scientst

In [ ]:
example = pd.read_csv('../../secret_data/Lab_vb_makeda.csv', sep=";")
example.head(4)

In [ ]:
example.columns

In [ ]:
example.shape

In [ ]:
# I reccomend turning K00 into K0, and NRCA to NRC0

### import baseline labs from 2022 and 2023
These labs should be named with _01 and uploaded to Castor as repeating data

In [ ]:
lab_data_baseline = pd.read_excel('../../secret_data/Brick_lab_20230914.xlsx')

In [ ]:
lab_data_baseline.columns

There is no particpant creation date!
You need to get it elsewhere


In [ ]:
lab_data_baseline.head(3)

In [ ]:
len(lab_data_baseline.studieID.unique())

In [ ]:
lab_data_baseline['studieID'] = lab_data_baseline['studieID'].astype(str)
lab_data_baseline['studieID'] = lab_data_baseline['studieID'].str.replace("-", "_")

In [ ]:
lab_data_baseline.head(3)

### We want  all the patients on thier own line, but not many lines per patient
We will try to accomplish this by splitting the dataset into each code-code-original i.e. each type of lab taken on the assumption we will almost always have one lab

In [ ]:
lab_data_baseline.code_code_original.unique()

In [ ]:
codes = lab_data_baseline.code_code_original.unique().astype(str)
codes

In [ ]:
#Here we must decide wif we want to switch nan to NA, or we will have no NA0 value in the end

In [ ]:
lab_data_baseline[lab_data_baseline['code_code_original'].isna()]

In [ ]:
lab_data_baseline['code_code_original'] = lab_data_baseline['code_code_original'].fillna('NA')

In [ ]:
codes = lab_data_baseline.code_code_original.unique().astype(str)
codes

In [ ]:
lab_data_baseline[lab_data_baseline['code_code_original'] == 'nan']

In [ ]:
len(codes)

In [ ]:
## Notte there is a code BRICK...let's examine

In [ ]:
lab_data_baseline[lab_data_baseline.code_code_original =='BRICK'].head(5)

In [ ]:
len(example.columns)

Scientists not interested in ICT, LIP, HEM, ORD8, BAF9 or EX_LUMC therefore we will drop these values

In [ ]:
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'ICT'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'LIP'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'HEM'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'ORD8'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'BAF9'].index)
lab_data_baseline = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['code_code_original']== 'EX_LUMC'].index)

Also according to example there are about 50 other codes they do ont care for. We will drop these here too: 

In [ ]:
codes0 = [code + "0" for code in codes]
codes0_set = set(codes0)
expected_set = set(example.columns)
expected_set

In [ ]:
# here we will show columsn we were essentially asked to get rid of
droppable = codes0_set - expected_set
list_droppable = list(droppable)
list_droppable = [code[:-1] for code in list_droppable]
#list_droppable

In [ ]:
# here we can show what we will keep
list_keepable = expected_set
list_keepable =  [code[:-1] for code in list_keepable]
list_keepable

In [ ]:
list_keepable_amended = ['CRP',
                         'FERT',
                         'TROMBO',
                         'MCV',
                         'K',
                         'FOLZ',
                         'HB',
                         'HT',
                         'UREU',
                         'VITD',
                         'GGT',
                         'ALB_U',
                         'SCHWARTZ-Bedsite',
                         'Participant Creation Date',
                         'ASAT',
                         'ERY',
                         'TBIL',
                         'DBIL',
                         'TE_U',
                         'TRAF',
                         'AFOS',
                         'NA',
                         'GRAT',
                         'LDH',
                         'FE',
                         'ALB_U_KR',
                         'B12',
                         'LEU',
                         'RETI',
                         'TSAT',
                         'KREA_U',
                         'ALAT',
                         'KREA',
                         'TE_U_KR',
                         'NRCA',
                         'RDW',]

In [ ]:
# lab_data_baseline = lab_data_baseline[~lab_data_baseline[lab_data_baseline['code_code_original'].isin(list_droppable)]]
# lab_data_baseline
lab_data_baseline = lab_data_baseline[lab_data_baseline['code_code_original'].isin(list_keepable_amended)]

In [ ]:
lab_data_baseline['code_code_original'].unique()

In [ ]:
new_codes = list(lab_data_baseline['code_code_original'].unique())

Scientist wants to get rid of doubles, i.e. brick 08, 12, 21 and 25. There are doubles because labs were not taken at precisely the same time 

In [ ]:
#example
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_008']['effectiveDateTime'].unique()

In [ ]:
#get the brick8 we want by resetting all the times to be the same
brick_8_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_008']#[lab_data['effectiveDateTime'] == '2022-08-29 10:05:12.0000000']
brick_8_wanted['effectiveDateTime'] = '2022-08-29 10:05:12.0000000'
#brick_8_wanted

In [ ]:
#len(brick_8_wanted)

In [ ]:
#len(brick_8_wanted.code_code_original.unique())

It did not work the way we wanted because there were repeats. Let's use only one date:

In [ ]:
brick_8_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_008'][lab_data_baseline['effectiveDateTime'] == '2022-08-29 10:05:12.0000000']
#brick_8_wanted 

In [ ]:
#12
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012']['effectiveDateTime'].unique()

In [ ]:
#lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012']

In [ ]:
# Here we will assume scientist wants the lab values 4 months later dropped, as not a baseline value
brick_12_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'][lab_data_baseline['effectiveDateTime'].isin(['2022-03-16 15:43:08', '2022-03-16 16:07:41'])]
# we will reset to all the same date
#brick_12_wanted 
brick_12_wanted['effectiveDateTime'] = '2022-03-16 15:43:08'

In [ ]:
brick_12_urine = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'][lab_data_baseline['effectiveDateTime'].isin(['2022-07-18 10:02:25'])]

In [ ]:
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_021']['effectiveDateTime'].unique()

In [ ]:
#brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 08:47:57']

In [ ]:
brick_21_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_021']
#brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 08:46:36']
#brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 08:47:57']
#brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 12:41:15']

In [ ]:
brick_21_urine = brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 12:41:15']

In [ ]:
brick_21_main = brick_21_wanted[brick_21_wanted['effectiveDateTime'] =='2023-06-29 08:47:57']

In [ ]:
brick_21_wanted = pd.concat([brick_21_urine, brick_21_main])
#brick_21_wanted

In [ ]:
## here we will "redate" them all to match
####brick_21_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_021']
brick_21_wanted['effectiveDateTime'] = '2023-06-29 08:46:36'
#brick_12_wanted

In [ ]:
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_025']['effectiveDateTime'].unique()

In [ ]:
#brick_25_wanted 
brick_25_mcv_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_025'][lab_data_baseline['effectiveDateTime'] == '2022-09-12 15:52:42.0000000']
#brick_25_mcv_wanted

In [ ]:
brick_25_urine_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_025'][lab_data_baseline['effectiveDateTime'] == '2022-09-12 15:44:02.0000000']
print(brick_25_urine_wanted.shape)
#brick_25_urine_wanted.

In [ ]:
#get the brick25 we want by concatenating and  resetting all the times to be the same
brick_25_wanted = pd.concat([brick_25_mcv_wanted ,brick_25_urine_wanted ])
print(brick_25_wanted.shape)
brick_25_wanted['effectiveDateTime'] = '2022-09-12 15:52:42'
brick_25_wanted.effectiveDateTime.unique()

In [ ]:
#12
lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012']['effectiveDateTime'].unique()

In [ ]:
# # Here we will assume scientist wants the lab values 4 months later dropped, as not a baseline value
# brick_12_wanted = lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'][lab_data_baseline['effectiveDateTime'].isin(['2022-03-16 15:43:08', '2022-03-16 16:07:41'])]
# # we will reset to all the same date
# #brick_12_wanted 
# brick_12_wanted['effectiveDateTime'] = '2022-03-16 15:43:08'

In [ ]:


#now let's drop all of them, then append back on what we want
lab_data_no_ = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_012'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_008'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_021'].index)
lab_data_no_ = lab_data_no_.drop(lab_data_no_[lab_data_no_['studieID'] == 'BRICK_025'].index)
#lab_data = 

In [ ]:
lab_data_no_.studieID.unique()

In [ ]:
#lab_data_baseline

In [ ]:
# lab_data2_no_ = lab_data_baseline.drop(lab_data_baseline[lab_data_baseline['studieID'] == 'BRICK_008'].index)
# lab_data2_no_ = lab_data2_no_.drop(lab_data2_no[lab_data2_no_['studieID'] == 'BRICK_012'].index)
# lab_data2_no_ = lab_data2_no_.drop(lab_data2_no[lab_data2_no_['studieID'] == 'BRICK_021'].index)
# lab_data2_no_ = lab_data2_no_.drop(lab_data2_no[lab_data2_no_['studieID'] == 'BRICK_025'].index)

In [ ]:
brick_12_urine.shape

In [ ]:
brick_12_wanted.shape

In [ ]:
lab_data = pd.concat([lab_data_no_, brick_8_wanted,  brick_12_wanted, brick_25_wanted, brick_21_wanted])
lab_data.shape

In [ ]:
list_of_data_frames = []
for code in new_codes:
    prenames = ["code_code_original", code]
    frame_name = "_".join(prenames)
    #print(frame_name)
    frame_name = lab_data[lab_data['code_code_original'] == code]
    frame_name = frame_name.add_suffix(code)
    frame_name = frame_name.rename(columns= {frame_name.columns[0]:'studieID'})
    list_of_data_frames.append(frame_name)
    

In [ ]:
len(list_of_data_frames)

In [ ]:
len(new_codes)

Let's look at an example:

In [ ]:
list_of_data_frames[0].head(3) 

In [ ]:
list_of_data_frames[1].head(3) 

In [ ]:
list_of_data_frames[1].shape

So far so good, we have a frame with in the example above the DBIL. We then tried to do a big reduction; but it produced repeaters

In [ ]:
semi_final_df = reduce(lambda left,right: pd.merge(left,right, on=['studieID'], how='outer'), list_of_data_frames)

In [ ]:
semi_final_df.shape

In [ ]:
semi_final_df.head(5)

In [ ]:
for studieID in semi_final_df.studieID:
    print(studieID)

In [ ]:
# now we need to change the columns names to those expoected by scientist
# for  f in semi_final_df.columns:
#     print(f)

In [ ]:
list_keepable_amended_with_id = ["valueQuantity_value" + code for code in list_keepable_amended]
# [code + "0" for code in codes]
list_keepable_amended_with_id = list_keepable_amended_with_id + ['studieID'] + ['Participant Creation Date']
#list_keepable_amended_with_id

In [ ]:
final_amend = ['studieID',
               #'Participant Creation Date',
               'valueQuantity_valueCRP',
                 'valueQuantity_valueFERT',
                 'valueQuantity_valueTROMBO',
                 'valueQuantity_valueMCV',
                 'valueQuantity_valueK',
                 'valueQuantity_valueFOLZ',
                 'valueQuantity_valueHB',
                 'valueQuantity_valueHT',
                 'valueQuantity_valueUREU',
                 'valueQuantity_valueVITD',
                 'valueQuantity_valueGGT',
                 'valueQuantity_valueALB_U',
                 'valueQuantity_valueSCHWARTZ-Bedsite',
                 'valueQuantity_valueASAT',
                 'valueQuantity_valueERY',
                 'valueQuantity_valueTBIL',
                 'valueQuantity_valueDBIL',
                 'valueQuantity_valueTE_U',
                 'valueQuantity_valueTRAF',
                 'valueQuantity_valueAFOS',
                 'valueQuantity_valueNA',
                 'valueQuantity_valueGRAT',
                 'valueQuantity_valueLDH',
                 'valueQuantity_valueFE',
                 'valueQuantity_valueALB_U_KR',
                 'valueQuantity_valueB12',
                 'valueQuantity_valueLEU',
                 'valueQuantity_valueRETI',
                 'valueQuantity_valueTSAT',
                 'valueQuantity_valueKREA_U',
                 'valueQuantity_valueALAT',
                 'valueQuantity_valueKREA',
                 'valueQuantity_valueTE_U_KR',
                 'valueQuantity_valueNRCA',
                 'valueQuantity_valueRDW',]

In [ ]:
filtered = semi_final_df[final_amend]
filtered

In [ ]:
columns_dictionary = {'studieID':'Participant ID',
                      'valueQuantity_valueCRP':'CRP0',  
                      'valueQuantity_valueFERT':'FERT0',
                      'valueQuantity_valueTROMBO':'TROMBO0',
                      'valueQuantity_valueMCV':'MCV0',
                      'valueQuantity_valueK':'K0',
                      'valueQuantity_valueFOLZ':'FOLZ0',
                      'valueQuantity_valueHB':'HB0',  
                      'valueQuantity_valueHT':'HT0',
                      'valueQuantity_valueUREU':'UREU0',  
                      'valueQuantity_valueVITD':'VITD0',
                      'valueQuantity_valueGGT':'GGT0',  
                      'valueQuantity_valueALB_U':'ALB_U0',
                      'valueQuantity_valueSCHWARTZ-Bedsite':'SCHWARTZ-Bedsite0',  
                      'valueQuantity_valueASAT':'ASAT0',
                      'valueQuantity_valueERY':'ERY0',  
                      'valueQuantity_valueTBIL':'TBIL0',
                      'valueQuantity_valueDBIL':'DBIL0',  
                      'valueQuantity_valueTE_U':'TE_U0',
                      'valueQuantity_valueTRAF':'TRAF0',  
                      'valueQuantity_valueNA':'NA0',
                      'valueQuantity_valueAFOS':'AFOS0',
                      'valueQuantity_valueGRAT':'GRAT0',  
                      'valueQuantity_valueLDH':'LDH0',
                      'valueQuantity_valueFE':'FE0',  
                      'valueQuantity_valueALB_U_KR':'ALB_U_KR0',
                      'valueQuantity_valueB12':'B120',  
                      'valueQuantity_valueLEU':'LEU0',
                      'valueQuantity_valueRETI':'RETI0',  
                      'valueQuantity_valueTSAT':'TSAT0',
                      'valueQuantity_valueKREA_U':'KREA_U0',  
                      'valueQuantity_valueALAT':'ALAT0',
                      'valueQuantity_valueKREA':'KREA0',  
                      'valueQuantity_valueTE_U_KR':'TE_U_KR0',
                      'valueQuantity_valueNRCA':'NRCA0',  
                      'valueQuantity_valueRDW':'RDW0'}
filtered = filtered.rename(columns = columns_dictionary)
filtered

In [ ]:
# Now we will do a remap + reduction on brick_12_urine
twelve_list_of_data_frames = []
for code in new_codes:
    prenames = ["code_code_original", code]
    frame_name = "_".join(prenames)
    #print(frame_name)
    frame_name = brick_12_urine[brick_12_urine['code_code_original'] == code]
    frame_name = frame_name.add_suffix(code)
    frame_name = frame_name.rename(columns= {frame_name.columns[0]:'studieID'})
    twelve_list_of_data_frames.append(frame_name)
    

In [ ]:
semi_final_twelve = reduce(lambda left,right: pd.merge(left,right, on=['studieID'], how='outer'), twelve_list_of_data_frames)
semi_final_twelve = semi_final_twelve[final_amend]
# semi_final_twelve_filtered = semi_final_twelve.rename(columns = columns_dictionary)
# semi_final_twelve_filtered['ALB_U0']

In [ ]:
semi_final_twelve = semi_final_twelve.rename(columns = columns_dictionary)
semi_final_twelve.columns

In [ ]:
semi_final_twelve

In [ ]:
filtered.columns

In [ ]:
filtered['Participant Status']= 'Enrolled'

In [ ]:
filtered['Site Abbreviation'] = 'ERA'

No idea where to get Participant Creation Date from ...it isn'tteh same as lab date, so let's leave it out.

In [ ]:
filtered.shape

In [ ]:
filtered_plus = pd.concat([filtered,semi_final_twelve])
filtered_plus

In [ ]:
filtered_plus.to_csv('../../secret_data/castor_with_salt_baseline_labs.csv')